In [8]:
import math
import torch
import gpytorch
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


from GPyEm import GPE_ensemble as GPE

import os
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

seed = 7
#set_seed(seed)

# User input: choose mesh

In [2]:
mesh=13

Load data

In [3]:
x_labels=pd.read_csv(r'/Users/pmzcwl/Library/CloudStorage/OneDrive-TheUniversityofNottingham/shared_simulations/EP/scenario_'+str(mesh)+'/xlabels.txt',delim_whitespace=True,header=None)


X_all = pd.read_csv(r'/Users/pmzcwl/Library/CloudStorage/OneDrive-TheUniversityofNottingham/shared_simulations/EP/scenario_'+str(mesh)+'/X.txt',index_col=None,delim_whitespace=True,header=None)
X_all.columns=x_labels.values.squeeze().tolist()
print(X_all)

     CV_ventricles  k_ventricles    k_FEC  CV_atria   k_atria     k_BB
0         0.738144      0.276378  5.71074  0.337617  0.253067  3.34120
1         0.699821      0.289857  8.23437  0.899943  0.244791  3.36932
2         0.419685      0.310723  1.32123  1.023790  0.208773  3.22631
3         0.780850      0.326027  4.83990  0.621390  0.201804  3.25724
4         0.794527      0.395183  5.83261  0.971377  0.346727  5.50506
..             ...           ...      ...       ...       ...      ...
995       0.639511      0.285979  6.68166  0.353444  0.207574  1.25594
996       0.487964      0.201533  2.99732  0.441108  0.371365  4.35580
997       0.506433      0.380463  6.07463  0.967278  0.359592  1.50839
998       0.686527      0.255578  2.70725  0.322308  0.303793  1.98914
999       0.745125      0.288781  6.70721  0.585251  0.369692  1.75873

[1000 rows x 6 columns]


In [4]:
y_labels=pd.read_csv(r'/Users/pmzcwl/Library/CloudStorage/OneDrive-TheUniversityofNottingham/shared_simulations/EP/scenario_13/ylabels.txt',delim_whitespace=True,header=None)


y_all = pd.read_csv(r'/Users/pmzcwl/Library/CloudStorage/OneDrive-TheUniversityofNottingham/shared_simulations/EP/scenario_'+str(mesh)+'/Y.txt',index_col=None,delim_whitespace=True,header=None)
y_all.columns=y_labels.values.squeeze().tolist()
print(y_all)

        A_TAT     V_TAT
0    315.4700   97.6231
1    119.7940  100.6130
2    112.3490  183.1760
3    187.0580   86.8046
4     90.6858   78.7504
..        ...       ...
175   90.0410   92.6852
176  147.1390   99.3163
177  143.7290  119.9870
178  144.0190  111.9080
179  166.9070  193.3910

[180 rows x 2 columns]


# Emulator

Convert pandas to torch

In [5]:
y_train = torch.tensor(y_all.values)
# Choose appropriate values from X_all (if X_all bigger than y_all)
x_train = torch.tensor(X_all[0:y_all.shape[0]].values) 

In [6]:
emulator = GPE.ensemble(x_train,y_train,mean_func="linear",training_iter=500) #Train emulator

0
1


In [7]:
# Generate max and min bounds
nDim = input_masked.shape[1]
boundsMaxMin = []
for i in range(nDim):
    boundsMaxMin.append([np.min(input_masked.iloc[:,i]),np.max(input_masked.iloc[:,i])])
    print(boundsMaxMin[i])

NameError: name 'input_masked' is not defined

In [ ]:
##### from SALib.sample import saltelli
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
import numpy as np

# Define the model inputs
problem = {
    'num_vars': nDim,
    'names': x_labels,
    'bounds': boundsMaxMin
    #'bounds': [[-1, 1],
     #          [-1, 1],
      #         [-1, 1],
       #        [-1, 1],
        #       [-1, 1],
         #      [-1, 1],
          #     [-1, 1],
           #    [-1, 1],
            #   [-1,1]]
}

# Generate samples
param_values = saltelli.sample(problem, 1024)

data1 = torch.tensor(param_values)
#data1.columns = X_train.columns

Ymean=emulator.predict(data1)

In [ ]:
data1.shape

In [ ]:
Si = []
nMod = y_all.shape[1]
nDim = input_masked.shape[1]
for i in range(nMod):
    Si.append(sobol.analyze(problem, Ymean[:,i].detach().numpy(), print_to_console=True,calc_second_order=True))

In [ ]:
Si_first=np.zeros((nDim,nMod))
Si_total=np.zeros((nDim,nMod))
for i in range(nMod):
    total_Si, first_Si, second_Si = Si[i].to_df()
    Si_first[:,i] =  first_Si.iloc[:,0]
    Si_total[:,i] =  total_Si.iloc[:,0]


In [ ]:
inputs = X_all.columns.tolist()
outputs = y_all.columns.tolist()
fig, ax = plt.subplots()
im = ax.imshow(Si_first[0:28],cmap='magma_r',vmin=0,vmax=1)

ax.set_xticks(np.arange(len(outputs)),labels=outputs)
ax.set_yticks(np.arange(len(inputs)), labels=inputs)
plt.setp(ax.get_xticklabels(), rotation=60, ha="right",
         rotation_mode="anchor");

plt.colorbar(im,fraction=0.015, pad=0.04)

In [ ]:
inputs = X_all.columns.tolist()
outputs = y_all.columns.tolist()
fig, ax = plt.subplots()
im = ax.imshow(Si_total.T[0:28],cmap='magma_r',vmin=0,vmax=1)

ax.set_yticks(np.arange(len(outputs)),labels=outputs)
ax.set_xticks(np.arange(len(inputs)), labels=inputs)
plt.setp(ax.get_xticklabels(), rotation=65, ha="right",
         rotation_mode="anchor");

plt.colorbar(im,fraction=0.015, pad=0.04)

In [ ]:
np.savetxt("Si_total_EP_"+str(mesh)+".csv", Si_total, delimiter=",")
np.savetxt("Si_first_EP_"+str(mesh)+".csv", Si_first, delimiter=",")